# Housing and Pop Forecasts 15County 2020-2050 

## Setup

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import time
from arcgis.features import FeatureLayerCollection
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile
import pandas as pd

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\dask\dataframe\__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


### Output/input pathing

In [2]:
# Load the uploaded file
INPUT_DATA_PATH = 'inputData/CountyControls_WEB_allData_Long.csv'

# Directory where images are saved
output_dir = 'outputData/plots'
os.makedirs(output_dir, exist_ok=True)

### Plot colors/paramters

In [3]:
# Dictionary to store COUNTY to itemID mapping to update insights catalog
county_item_mapping = {}

## Pulling Existing Catalog from AGO

In [4]:
# Define the directory to save the data
data_path = Path('inputData/insightsCatalog')
data_path.mkdir(parents=True, exist_ok=True)

# Access the CSV item by ID
catalog_item_id = "c396b9d0db724d1f870fa47f1d1dc1ce"
csv_item = gis.content.get(catalog_item_id)

# Download the CSV file
catalog_path = csv_item.download(save_path=data_path)

print(f"Excel file downloaded successfully: {catalog_path}")

Excel file downloaded successfully: inputData\insightsCatalog\morpc_insights_catalog.xlsx


## Creating plots for 15 counties and region summary

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load the dataset
file_path = 'inputData/CountyControls_WEB_allData_Long.csv'
df = pd.read_csv(file_path)

# Define relevant variables
relevant_variables = [
    'Household Population Under 18', 'Population Ages 18 - 64',
    'Household Population 65 and Older', 'Households', 'Housing Units', 
    'Workers', 'Labor Force'
]

# Filter data based on specified conditions
filtered_df = df[
    (df['VARIABLE'].isin(relevant_variables)) & 
    (df['COUNTY'] != '10-County Region')
]


# Define color palette
colors = ['#365072', '#0075BF', '#00B2BF', '#2A7F67', '#8DBF00', '#A5BF00', '#4F8FC0']

# Get unique county names after filtering
unique_counties = filtered_df['COUNTY'].unique()

# Generate line plots with 16:9 aspect ratio and updated font size
for county in unique_counties:
    subset = filtered_df[filtered_df['COUNTY'] == county]

    # Pivot the data for easier plotting
    pivot_data = subset.pivot(index='Year', columns='VARIABLE', values='Value')

    plt.figure(figsize=(16, 9))  # Set aspect ratio to 16:9
    for i, col in enumerate(pivot_data.columns):
        plt.plot(pivot_data.index, pivot_data[col], marker='o', label=col, color=colors[i % len(colors)])

    plt.title(f"Population and Housing Forecast in {county} County (2020-2050)", fontsize=18)

    # Place legend below the plot
    plt.legend(fontsize=15, loc='lower center', bbox_to_anchor=(0.5, -0.3), ncol=2, frameon=True)

    plt.grid(True)
    
    # Remove x and y axis labels
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    # Save each plot in the created directory
    filename = os.path.join(output_dir, f"{county.replace(' ', '_')}_population_predictions.png")
    plt.savefig(filename, bbox_inches='tight')  # Ensure legend is fully captured
    plt.close()

print(f"Updated plots saved in the '{output_dir}' directory with 16:9 aspect ratio and font size 15.")


Updated plots saved in the 'outputData/plots' directory with 16:9 aspect ratio and font size 15.


# Upload to AGO

In [6]:
import time
from arcgis.gis import GIS
import os
import pandas as pd

output_dir = 'outputData/plots'

# Initialize list to store county-item mappings
county_item_mapping = []

# Connect to ArcGIS Online
gis = GIS("https://www.arcgis.com", "ogwynn_morpc", "Cr33p3r11!!")

# Loop through the output directory and upload files
for filename in os.listdir(output_dir):
    if filename.endswith(".png"):
        file_path = os.path.join(output_dir, filename)
        
        # Define item properties
        title = filename.replace("_", " ").replace(".png", "")
        county_name = filename.split('_')[0]  # Extract county name from the filename
        item_properties = {
            "title": title,
            "description": f"Population and Housing Forecast in {county_name} County (2020-2050).",
            "type": "Image"
        }
        
        # Search for an existing item with the same title
        existing_items = gis.content.search(query=title, item_type="Image", max_items=1)
        if existing_items:
            # If an existing item is found, delete it
            existing_item = existing_items[0]
            existing_item.delete()
            print(f"Deleted existing item with title: {title}")
            time.sleep(2)  # Wait for 2 seconds before uploading the new item

        try:
            # Upload the new image
            image_item = gis.content.add(item_properties, data=file_path)
            
            # Make the item public
            image_item.share(everyone=True)

            # Append the county name, item URL, and title to the list
            county_item_mapping.append((county_name, f"https://www.arcgis.com/sharing/rest/content/items/{image_item.id}/data", title))

            # Print confirmation
            print(f"Uploaded {filename} with item ID: {image_item.id} and set it to public")
        
        except Exception as e:
            print(f"Error uploading {filename}: {e}")

# Convert the list to a DataFrame and save
mapping_df = pd.DataFrame(county_item_mapping, columns=['COUNTY', 'itemURL', 'title'])
mapping_file_path = 'AGO_County_ItemURL_Mapping.csv'
mapping_df['TilesetID'] = 12
mapping_df.to_csv(mapping_file_path, index=False)

print(f"Upload complete. Mapping file saved as '{mapping_file_path}'.")


Deleted existing item with title: 15-County Region population predictions


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded 15-County_Region_population_predictions.png with item ID: b14467f97d074c4dae0947ad3858b4b6 and set it to public
Deleted existing item with title: Delaware population predictions


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Delaware_population_predictions.png with item ID: e9e3cff855464e149703ce11e4e313fc and set it to public
Deleted existing item with title: Fairfield population predictions


...

Uploaded Pickaway_population_predictions.png with item ID: 6dd6a0017e954b258ad181183c0e5732 and set it to public
Deleted existing item with title: Ross population predictions


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Ross_population_predictions.png with item ID: 2934eed2227847a895052761632202fa and set it to public
Deleted existing item with title: Union population predictions


C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: add is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Folder.add()` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\ogwynn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\interactiveshell.py:3577: DeprecatedWarning: share is deprecated as of 2.3.0 and has been removed in 3.0.0. Use `Item.sharing` instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


Uploaded Union_population_predictions.png with item ID: 3f149e8b2c544493bddaab6151b3d4f4 and set it to public
Upload complete. Mapping file saved as 'AGO_County_ItemURL_Mapping.csv'.


## Updating local catalog

In [7]:
# Load the catalog data
catalog_df = pd.read_excel(catalog_path)
catalog_df.head()

,TileID,TilesetID,GeographyType,GeographyName,Category,Headline,Commentary,ThumbnailURL,Contributor,Vintage,UpdateInterval,ShareURL,DataProductURL,MoreInformationURL
0,1,1,Region,Central Ohio,Demographics,Strong and Steady Growth Projected for Central...,The 15-county <b> Central Ohio </b> region is ...,https://morpc.maps.arcgis.com/sharing/rest/con...,MORPC Data Team,2023,4 years,NaN,https://www.morpc.org/wordpress/wp-content/upl...,NaN
1,2,2,Region,Central Ohio,Housing,Major Developments Will Have Widespread Impacts,Communities must collaborate with neighboring ...,https://morpc.maps.arcgis.com/sharing/rest/con...,MORPC Community Development Team,2023,Ad hoc,NaN,https://www.morpc.org/2023/wp-content/uploads/...,NaN
2,3,3,Region,Central Ohio,Wellness,Central Ohio Legacy Cities Foster Walkable Nei...,Ohio's legacy cities have many highly-sought f...,https://morpc.maps.arcgis.com/sharing/rest/con...,MORPC Data Team,2023,4 years,NaN,https://cura.osu.edu/events/strongtowns,NaN
3,4,4,Region,Central Ohio,Housing,Central Ohio Households On Track to Significan...,The number of households in the region is expe...,https://morpc.maps.arcgis.com/sharing/rest/con...,MORPC Data Team,2023,4 years,NaN,https://www.morpc.org/2023/wp-content/uploads/...,NaN
4,5,5,Region,Central Ohio,Transportation,Bicycle Involved Crashes and Fatalities Remain...,Dashboard data from the Ohio Department of Tra...,https://morpc.maps.arcgis.com/sharing/rest/con...,Ohio Department of Transportation,May 2023,Monthly,NaN,https://www.transportation.ohio.gov/programs/h...,NaN


In [8]:
import pandas as pd

# Load the catalog data
catalog_df = pd.read_excel(catalog_path)

# Load the mapping data
mapping_df = pd.read_csv(mapping_file_path)

# Standardize 'COUNTY' values in mapping_df
mapping_df['COUNTY'] = mapping_df['COUNTY'].replace('15-County', '15County')

# Create a mapping dictionary from mapping_df
mapping_dict = dict(zip(zip(mapping_df['COUNTY'], mapping_df['TilesetID']), mapping_df['itemURL']))

# Update 'ThumbnailURL' only where there is a match in mapping_dict
catalog_df['ThumbnailURL'] = catalog_df.apply(
    lambda row: mapping_dict.get((row['GeographyName'], row['TilesetID']), row['ThumbnailURL']),
    axis=1
)

# Save the updated catalog back to an Excel file
updated_catalog_path = "outputData/morpc_insights_catalog_updated.xlsx"
catalog_df.to_excel(updated_catalog_path, index=False)

# Provide the file to the user
print(f"Updated catalog saved at: {updated_catalog_path}")


Updated catalog saved at: outputData/morpc_insights_catalog_updated.xlsx


## Pushing changes to AGO Excel

In [9]:
# Define the item ID of the Excel file in AGO
ITEM_ID = "c396b9d0db724d1f870fa47f1d1dc1ce"

# Path to the updated Excel file
updated_catalog_path = "outputdata/morpc_insights_catalog_updated.xlsx"

# Get the existing item
item = gis.content.get(ITEM_ID)

if item:
    print(f"Found item: {item.title}")

    # Update the item with the new file
    item.update(
        item_properties={"title": item.title},  # Keeps the existing metadata
        data=updated_catalog_path  # Replaces the file
    )
    
    print("Successfully updated the Excel file in AGO.")
else:
    print("Item not found. Check the item ID and your permissions.")


Found item: morpc_insights_catalog
Successfully updated the Excel file in AGO.
